# 决策树

- ID3（基于信息增益）
- C4.5（基于信息增益比）
- CART（gini指数）

#### entropy：$H(x) = -\sum_{i=1}^{n}p_i\log{p_i}$

#### conditional entropy: $H(X|Y)=\sum{P(X|Y)}\log{P(X|Y)}$

#### information gain : $g(D, A)=H(D)-H(D|A)$

#### information gain ratio: $g_R(D, A) = \frac{g(D,A)}{H(A)}$

#### gini index:$Gini(D)=\sum_{k=1}^{K}p_k\log{p_k}=1-\sum_{k=1}^{K}p_k^2$

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

## 节点类

In [2]:
class Node:
    def __init__(self, x=None, label=None, y=None, data=None):
        # 特征值序号
        self.label = label   
        # x为特征值的类别值
        self.x = x
        # 子节点
        self.child = []    
        # 分类类别
        self.y = y
        # 分类类别数据
        self.data = data   
    
    # 添加子节点
    def append(self, node):  
        self.child.append(node)

    # 根据决策树进行预测
    def predict(self, features): 
        if self.y is not None:
            # 返回预测结果
            return self.y
        # 对子节点进行遍历
        for c in self.child:
            # 特征值x为待预测数据的特征值
            if c.x == features[self.label]:
                return c.predict(features)

## 节点打印函数

In [3]:
def printnode(node, depth=0):  
    if node.label is None:
        print(depth, (node.label, node.x, node.y, len(node.data)))
    else:
        print(depth, (node.label, node.x))
        for c in node.child:
            printnode(c, depth+1)

In [4]:
class DTree:
    def __init__(self, epsilon=0, alpha=0): 
        # 阈值
        self.epsilon = epsilon
        # alpha值
        self.alpha = alpha
        # 初始化为单节点树
        self.tree = Node()
    
    # 计算概率值
    def prob(self, datasets): 
        # 得到数据集的样本总数
        datalen = len(datasets)
        # 得到样本的分类类别数
        labelx = set(datasets)
        # 初始化频值
        p = {l: 0 for l in labelx}
        # 统计各个频值数
        for d in datasets:
            p[d] += 1
        # 得到每个类别的概率值
        for i in p.items():
            p[i[0]] /= datalen
        return p
    
    # 计算熵
    def calc_ent(self, datasets): 
        p = self.prob(datasets)
        ent = sum([-v * math.log(v, 2) for v in p.values()])
        return ent

    # 计算经验条件熵
    def cond_ent(self, datasets, col):  
        # 取出第col行的数据（即取出第几个特征值对应的行数据）
        labelx = set(datasets.iloc[col])
        # 初始化特征值对应的频值字典
        p = {x: [] for x in labelx}
        # 对i(第i条数据)，d(分类类别)迭代
        # 统计特征值下的各个类别的标签分类
        for i, d in enumerate(datasets.iloc[-1]):
            p[datasets.iloc[col][i]].append(d)
        # 计算经验条件熵
        return sum([self.prob(datasets.iloc[col])[k] * self.calc_ent(p[k]) for k in p.keys()])

    def info_gain_train(self, datasets, datalabels):
        # 将数据集转置，即特征值是表的RowName，数据集是0-14列
        datasets = datasets.T
        # iloc[num]：通过行号来取行数据
        # 取最后一行数据（即类别行）计算经验熵
        ent = self.calc_ent(datasets.iloc[-1])
        # 初始化信息增益字典
        gainmax = {}
        # 计算信息增益，并将信息增值作为字典的key，而value为数据序号
        # len(datasets) - 1 除去分类列
        for i in range(len(datasets) - 1):
            cond = self.cond_ent(datasets, i)
            gainmax[ent - cond] = i
        # 取最大信息增益
        m = max(gainmax.keys())
        # gainmax[m]：最大信息增益对应的特征值序号，m：信息增益
        return gainmax[m], m

    # 训练函数（节点划分函数）
    # datasets：数据集，node：节点树
    def train(self, datasets, node):
        # 取标签列
        labely = datasets.columns[-1]
        # value_counts() 以Series形式返回指定列的不同取值的频率
        # 如果都为同一标签分类的数据
        if len(datasets[labely].value_counts()) == 1:
            # 该子节点的数据集为该分类列的数据
            node.data = datasets[labely]
            # y为标签分类类别
            node.y = datasets[labely][0]
            return
        # 如果没有了需要分类的特征值
        if len(datasets.columns[:-1]) == 0:
            # 该节点的数据集为该分类列的数据
            node.data = datasets[labely]
            # y为该标签分类的类别
            node.y = datasets[labely].value_counts().index[0]
            return
        
        # 得到gainmaxi：最优的特征，gainmax：最大信息增益
        gainmaxi, gainmax = self.info_gain_train(datasets, datasets.columns)
        # 小于阈值，不需要再划分了
        if gainmax <= self.epsilon: 
            node.data = datasets[labely]
            node.y = datasets[labely].value_counts().index[0]
            return
        
        # 最优特征下的类别频值
        vc = datasets[datasets.columns[gainmaxi]].value_counts()
        # 对类别频值进行遍历
        for Di in vc.index:
            # label赋值为最优特征值序号
            node.label = gainmaxi
            # 创建类别子节点，x为类别值
            child = Node(Di)
            # 添加子节点列表到类别节点
            node.append(child)
            # 得到该特征值下为Di类别的数据集
            new_datasets = pd.DataFrame([list(i) for i in datasets.values if i[gainmaxi]==Di], columns=datasets.columns)
            # 对数据集进行再分类
            self.train(new_datasets, child)
    
    # 匹配函数
    def fit(self, datasets):
        self.train(datasets, self.tree)
    
    # 得到所有的叶子节点集合
    def findleaf(self, node, leaf): 
        # 对该节点下的子节点进行遍历
        for t in node.child:
            # 判断分类值是否为空，如果为空，则表示这是一个特征，如果不为空，则表示这是一个叶子节点
            if t.y is not None:
                # 添加数据到叶子节点集合中
                leaf.append(t.data)
            else:
                for c in node.child:
                    self.findleaf(c, leaf)

    def findfather(self, node, errormin):
        if node.label is not None:
            cy = [c.y for c in node.child]
            if None not in cy:  
                childdata = []
                for c in node.child:
                    for d in list(c.data):
                        childdata.append(d)
                # 统计childdata的频值
                childcounter = Counter(childdata)

                old_child = node.child  
                old_label = node.label
                old_y = node.y
                old_data = node.data

                node.label = None  
                # 根据奥卡姆剃刀准则，进行简化处理
                node.y = childcounter.most_common(1)[0][0]
                node.data = childdata
                
                #再次计算预测误差
                error = self.c_error()
                # 获得最小的预测误差
                if error <= errormin:  
                    errormin = error
                    return 1
                else:
                    node.child = old_child  
                    node.label = old_label
                    node.y = old_y
                    node.data = old_data
            else:
                re = 0
                i = 0
                while i < len(node.child):
                    if_re = self.findfather(node.child[i], errormin)  
                    if if_re == 1:
                        re = 1
                    elif if_re == 2:
                        i -= 1
                    i += 1
                if re:
                    return 2
        return 0
    
    # 求模型对训练数据的预测误差
    def c_error(self): 
        leaf = []
        # 找到叶子节点
        self.findleaf(self.tree, leaf)
        # 计算每个特征值下的总数
        leafnum = [len(l) for l in leaf]
        # 计算每一类的信息熵
        ent = [self.calc_ent(l) for l in leaf]
        print("Ent:", ent)
        # 求偏差alpha*|T|
        error = self.alpha*len(leafnum)
        # 求损失函数的C(T)值
        for l, e in zip(leafnum, ent):
            error += l*e
        print("C(T):", error)
        return error
    
    # 剪枝函数
    def cut(self, alpha=0): 
        if alpha:
            self.alpha = alpha
        # 得到预测误差值
        errormin = self.c_error()
        self.findfather(self.tree, errormin)

## 初始化数据集

In [5]:
datasets = np.array([['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否']])

## 得到训练数据集和预测数据

In [6]:
datalabels = np.array(['年龄', '有工作', '有自己的房子', '信贷情况', '类别'])
train_data = pd.DataFrame(datasets, columns=datalabels)
test_data = ['老年', '否', '否', '一般']

## 对训练数据集训练得到决策树

In [7]:
dt = DTree(epsilon=0) 
dt.fit(train_data)

## 打印决策树

In [8]:
print('DTree:')
printnode(dt.tree)
y = dt.tree.predict(test_data)
print('result:', y)

DTree:
0 (2, None)
1 (1, '否')
2 (None, '否', '否', 6)
2 (None, '是', '是', 3)
1 (None, '是', '是', 6)
result: 否


## 进行后剪枝操作

In [9]:
dt.cut(alpha=0.5)

Ent: [0.0, 0.0, 0.0]
C(T): 1.5
Ent: [0.9182958340544896, 0.0]
C(T): 9.264662506490406


In [10]:
print('DTree:')
printnode(dt.tree)
y = dt.tree.predict(test_data)
print('result:', y)

DTree:
0 (2, None)
1 (1, '否')
2 (None, '否', '否', 6)
2 (None, '是', '是', 3)
1 (None, '是', '是', 6)
result: 否


## 剪枝效果

In [11]:
datasets1 = np.array([['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '一般', '是']])

In [12]:
datalabels = np.array(['年龄', '有工作', '有自己的房子', '信贷情况', '类别'])
train_data = pd.DataFrame(datasets1, columns=datalabels)
test_data = ['老年', '否', '否', '一般']

dt = DTree(epsilon=0) 
dt.fit(train_data)

print('DTree:')
printnode(dt.tree)
y = dt.tree.predict(test_data)
print('result:', y)
print('----------------')
dt.cut(alpha=0.5)

print('----------------')
print('DTree:')
printnode(dt.tree)
y = dt.tree.predict(test_data)
print('result:', y)

DTree:
0 (2, None)
1 (1, '否')
2 (0, '否')
3 (3, '青年')
4 (None, '一般', '否', 3)
4 (None, '好', '否', 1)
3 (None, '中年', '否', 2)
3 (None, '老年', '否', 1)
2 (None, '是', '是', 3)
1 (None, '是', '是', 6)
result: 否
----------------
Ent: [0.9182958340544896, 0.0, 0.0, 0.0, 0.0, 0.0]
C(T): 5.754887502163468
Ent: [0.8112781244591328, 0.0, 0.0, 0.0, 0.0]
C(T): 5.745112497836532
Ent: [0.5916727785823275, 0.0, 0.0]
C(T): 5.641709450076292
Ent: [0.9709505944546686, 0.0]
C(T): 10.709505944546686
----------------
DTree:
0 (2, None)
1 (1, '否')
2 (None, '否', '否', 7)
2 (None, '是', '是', 3)
1 (None, '是', '是', 6)
result: 否
